In [1]:
from multiprocessing import Pool, cpu_count

import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm

from deep_mca.utils import disassemble_hex_to_block

/Users/henrycheng/Documents/CS172B/deep-mca/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("stevenhe04/x86-bb-24m")
df = dataset["train"].to_pandas()

df["hex"] = df["hex"].fillna("").astype(str).str.strip()
df = df[df["hex"] != ""].reset_index(drop=True)

In [3]:
with Pool(cpu_count()) as pool:
    results = list(
        tqdm(
            pool.imap(disassemble_hex_to_block, df["hex"], chunksize=512),
            total=len(df),
        )
    )

pretrain_x86 = pd.DataFrame(results, columns=["instructions", "valid"])
pretrain_x86.to_parquet("../data/pretrain_x86.parquet", index=False)

print(f"Total rows: {len(pretrain_x86)}")
print(f"Valid rows: {pretrain_x86['valid'].sum()}")
print(f"Invalid rows: {(~pretrain_x86['valid']).sum()}")

100%|██████████| 24442085/24442085 [16:37:00<00:00, 408.59it/s]   


Total rows: 24442085
Valid rows: 23048454
Invalid rows: 1393631
